Check Cluster State
====
This notebook check the state of ES cluster and sends mails to all the people substribed to that alert.
It is run once per hour from a cron job.

In [1]:
import subscribers, alerts
import requests

In [2]:
r=requests.get('http://cl-analytics.mwt2.org:9200/_cluster/health')
res=r.json()
print(res)

{'active_shards': 18645, 'unassigned_shards': 15, 'active_primary_shards': 13462, 'initializing_shards': 0, 'status': 'red', 'cluster_name': 'atlas', 'timed_out': False, 'number_of_nodes': 10, 'active_shards_percent_as_number': 99.91961414790997, 'task_max_waiting_in_queue_millis': 0, 'number_of_pending_tasks': 0, 'delayed_unassigned_shards': 0, 'relocating_shards': 0, 'number_of_data_nodes': 5, 'number_of_in_flight_fetch': 0}


In [7]:
if res['status'] == 'green':
    exit
    
S = subscribers.subscribers()
A = alerts.alerts()

if res['status'] == 'red':
    testName = 'Alert on Elastic cluster state [ES in red]'
    subscribersToRed =  S.get_immediate_subscribers(testName)
    for subscriber in subscribersToRed:
        body = 'Dear ' + subscriber.name +',\n\n'
        body = body + '\tthis mail is to let you that the University of Chicago Elasticsearch cluster is in RED.\n'
        body = body + '\nBest regards,\nATLAS AAS'
        body = body + '\n\n To change your alerts preferences please you the following link:\n' + subscriber.link
        print(subscriber.to_string())
        A.sendMail(testName, subscriber.email, body)
        A.addAlert(testName, subscriber.name,'simply red.')
if res['status'] == 'yellow' and res['unassigned_shards']>10:
    testName = 'Alert on Elastic cluster state [ES in yellow]'
    subscribersToYellow = S.get_immediate_subscribers(testName)
    for subscriber in subscribersToYellow:
        body = 'Dear ' + subscriber.name+',\n\n'
        body = body + '\tthis mail is to let you that the University of Chicago Elasticsearch cluster is in YELLOW.'
        body = body + 'There is '+str(res['unassigned_shards']) + ' unassigned shards on '+str(res['number_of_nodes']) + ' nodes.\n'
        body = body + '\nBest regards,\nATLAS AAS'
        body = body + '\n\n To change your alerts preferences please you the following link:\n'+subscriber.link
        print(subscriber.to_string())
        A.sendMail(testName, subscriber.email, body)
        A.addAlert(testName, subscriber.name, str(res['unassigned_shards']) + ' unassigned shards on '+str(res['number_of_nodes']) + ' nodes.' )
